In [11]:
script = """#!/bin/bash

#exit the job if there is something wrong with /scratch
cd /scratch || exit
mkdir -p {solisid} || exit
cd {solisid} || exit

echo "Copying model to /scratch..."

#recursively copy all the project files from homedir to node /scratch/solisid
rsync -r --info=progress2 /nethome/{solisid}/src .
cd src/TransmissionModel

echo "Installing requirements"
pip install -r requirements.txt


echo "Running models..."
for i in {{{loop_start}..{loop_end}}} ; do
  python3 __MainT__.py $i &
done

wait

echo "Completed models, copying results to /nethome/solisid..."

cd ..
cd Data
rsync -r --info=progress2 ./Model_V1/Data/High /nethome/{solisid}/results/{demo}

echo "Done..."
#end of job"""

# Generate jobs in batches of 16
for job in range(0, 24):
    job_start = job * 16
    job_end = min(job_start + 15, 380)
    job_script = script.format(loop_start=job_start, loop_end=job_end, demo=3, solisid=6958680)
    with open(f"jobs/job{job}.sh", "w") as file:
        file.write(job_script)
